In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder

import sys
import imp

sys.path.append('../src/')
import data.reader as dataReader
import utils.utils as thesisUtils
import similarities.cosine as thesisCosineSimilarity
import vocabulary.vocabulary as thesisVocabulary
import features.model_features as thesisModelFeatures
import features.factory as thesisFactoryFeatures
import data.corpus_stats as thesisCorpusStats

imp.reload(dataReader)
imp.reload(thesisUtils)
imp.reload(thesisVocabulary)
imp.reload(thesisCorpusStats)
imp.reload(thesisModelFeatures)
imp.reload(thesisFactoryFeatures)
imp.reload(thesisCosineSimilarity)

<module 'similarities.cosine' from '../src/similarities/cosine.py'>

In [3]:
london_corpus = dataReader.CorpusByNewLine.london()
zwickau_corpus = dataReader.CorpusByNewLine.zwickau()

In [4]:
np.average([ len("".join(i.split())) for i in london_corpus.corpus])

506.3659305993691

In [5]:
featuresFactory  = thesisFactoryFeatures.FeaturesFactory(
    london_corpus = london_corpus,
    zwickau_corpus = zwickau_corpus
)

In [6]:
london_zwickau_similarities = thesisCosineSimilarity.CrossVersionSimilarity5Gram(london_corpus, zwickau_corpus)
# london_zwickau_similarities.calculate()

zwickau_london_similarities = thesisCosineSimilarity.CrossVersionSimilarity5Gram(zwickau_corpus, london_corpus)
# zwickau_london_similarities.calculate()

In [7]:
# london_zwickau_similarities.save()
# zwickau_london_similarities.save()
london_zwickau_similarities.load()
zwickau_london_similarities.load()

In [8]:
burchard_corpus_by_london = dataReader.BurchardCorpus(london_corpus, zwickau_corpus)
burchard_corpus_by_zwickau = dataReader.BurchardCorpus(zwickau_corpus, london_corpus)

In [9]:
london_leftovers = dataReader.LeftoversCorpus(london_corpus, zwickau_corpus)
zwickau_leftovers = dataReader.LeftoversCorpus(zwickau_corpus, london_corpus)

In [10]:
print(len(london_leftovers.corpus_for_predictions()))
print(len(zwickau_leftovers.corpus_for_predictions()))

157
146


In [11]:
def filter_short_p(corpus):
    return list(filter(lambda x: len(x.split()) > 20, corpus))

In [12]:
def build_burchard_corpus_with_predictions(burchard_corpus, wrong_predictions_by_london, wrong_predictions_by_zwickau):
    is_burchard = True
    temp_corpus = [ [p, is_burchard, is_burchard] for p in burchard_corpus ]
    
    for prediction in wrong_predictions_by_london:
        temp_corpus[prediction.index][1] = False
    for prediction in wrong_predictions_by_zwickau:
        temp_corpus[prediction.index][2] = False

    return temp_corpus

In [13]:
london_leftofvers_long = filter_short_p(london_leftovers.corpus)
zwickau_leftofvers_long = filter_short_p(zwickau_leftovers.corpus)
burchard_corpus_by_london_corpus_long = filter_short_p(burchard_corpus_by_london.corpus)
burchard_corpus_by_zwickau_corpus_long = filter_short_p(burchard_corpus_by_zwickau.corpus)

In [14]:
print(len(burchard_corpus_by_london_corpus_long))
print(len(burchard_corpus_by_zwickau_corpus_long))

203
203


In [24]:
# burchard_by_london_VS_zwickau_features_df = thesisModelFeatures.create_features_df(
#     london_corpus = None,
#     zwickau_corpus = zwickau_leftofvers_long,
#     n_gram = (2,5),
#     features = { 'tfidf', 'inner_mean_cosine_similarity_score' }
# )
# burchard_by_zwickau_VS_zwickau_features_df = thesisModelFeatures.create_features_df(
#     london_corpus = None,
#     zwickau_corpus = zwickau_leftofvers_long,
#     n_gram = (2,5),
#     features = { 'tfidf', 'inner_mean_cosine_similarity_score' }
# )
# burchard_by_london_VS_london_features_df = thesisModelFeatures.create_features_df(
#     london_corpus = london_leftofvers_long,
#     zwickau_corpus = None,
#     n_gram = (2,5),
#     features = { 'tfidf', 'inner_mean_cosine_similarity_score' }
# )
# burchard_by_zwickau_VS_london_features_df = thesisModelFeatures.create_features_df(
#     london_corpus =london_leftofvers_long,
#     zwickau_corpus = None,
#     n_gram = (2,5),
#     features = { 'tfidf', 'inner_mean_cosine_similarity_score' }
# )

df = featuresFactory.burchard_by_london_VS_zwickau()

n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram


## Candidate VS zwickau

In [25]:
burchard_by_london_VS_zwickau_models_experiment = thesisModelFeatures.ModelsExperiment(df, 'big_hope')

In [26]:
burchard_by_london_VS_zwickau_models_experiment.run_cross_validate()

running: SVM_linear


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

Elapsed time to compute the cross validate: 0.518 minutes
running: SVM_RBF
Elapsed time to compute the cross validate: 0.575 minutes
running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.136 minutes
running: QuadraticDiscriminantAnalysis


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analy

Elapsed time to compute the cross validate: 0.907 minutes
running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.139 minutes
running: GaussianProcessClassifier
Elapsed time to compute the cross validate: 0.900 minutes
running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.121 minutes
running: MLPClassifier
Elapsed time to compute the cross validate: 4.767 minutes
running: GaussianNB
Elapsed time to compute the cross validate: 0.075 minutes
running: KNeighborsClassifier
Elapsed time to compute the cross validate: 0.067 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 0.694 minutes
running: XGBClassifier
Elapsed time to compute the cross validate: 2.154 minutes


In [27]:
burchard_by_london_VS_zwickau_models_experiment.show_cross_validate_results()

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
XGBClassifier,0.833020,0.808810,0.814038,0.825546,0.821450,0.825546
AdaBoostClassifier,0.780077,0.768690,0.770442,0.779664,0.777805,0.779664
RandomForestClassifier,0.736674,0.700119,0.700978,0.728151,0.715509,0.728151
DecisionTreeClassifier,0.710832,0.702976,0.699308,0.708067,0.706625,0.708067
SVM_RBF,0.750972,0.645714,0.629794,0.693613,0.654683,0.693613
GaussianProcessClassifier,0.792409,0.634167,0.608110,0.690756,0.637669,0.690756
MLPClassifier,0.642064,0.592619,0.580862,0.633697,0.604817,0.633697
KNeighborsClassifier,0.642882,0.550833,0.490248,0.616134,0.532242,0.616134
GaussianNB,0.602404,0.520476,0.457501,0.584454,0.499864,0.584454
SVM_linear,0.290840,0.500000,0.367719,0.581681,0.427924,0.581681


In [28]:
modelsToTest = [
    ( thesisModelFeatures.Model(RandomForestClassifier(max_depth = 9, n_estimators = 400, random_state = 0)), 'RandomForestClassifier', {} ),
    ( thesisModelFeatures.Model(AdaBoostClassifier(learning_rate = 1, n_estimators = 2000)), 'AdaBoostClassifier', {} ),
    ]

In [30]:
import time
scores_df = pd.DataFrame(dtype=float)
scoring = ('precision_macro', 'recall_macro', 'f1_macro', 'f1_micro', 'f1_weighted', 'accuracy')

cross_validate_results = []
for m, classifier_name, greed_search_cv_params in modelsToTest:
    print(f'running: {classifier_name}')
    start_time = time.time()
    classifier_cross_validate_score = m.run_cross_validate(df)
    elapsed_time = time.time() - start_time
    print(f"Elapsed time to compute the cross validate: {elapsed_time/60:.3f} minutes")
    cross_validate_results.append(classifier_cross_validate_score)
    for s in scoring:
        scores_df.loc[classifier_name, s] = classifier_cross_validate_score[f'test_{s}'].mean()

running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.268 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 25.489 minutes


In [31]:
scores_df

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
RandomForestClassifier,0.778396,0.718929,0.720251,0.751092,0.735244,0.751092
AdaBoostClassifier,0.868232,0.841190,0.847723,0.856891,0.853699,0.856891


In [37]:
modelsToTest = [
    ( thesisModelFeatures.Model(xgb.XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.4, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=5,
            monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1)), 'XGBClassifier', {} ),
    ]

In [38]:
import time
scores_df = pd.DataFrame(dtype=float)
scoring = ('precision_macro', 'recall_macro', 'f1_macro', 'f1_micro', 'f1_weighted', 'accuracy')

cross_validate_results = []
for m, classifier_name, greed_search_cv_params in modelsToTest:
    print(f'running: {classifier_name}')
    start_time = time.time()
    classifier_cross_validate_score = m.run_cross_validate(df)
    elapsed_time = time.time() - start_time
    print(f"Elapsed time to compute the cross validate: {elapsed_time/60:.3f} minutes")
    cross_validate_results.append(classifier_cross_validate_score)
    for s in scoring:
        scores_df.loc[classifier_name, s] = classifier_cross_validate_score[f'test_{s}'].mean()

running: XGBClassifier
Elapsed time to compute the cross validate: 1.936 minutes


In [39]:
scores_df

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
XGBClassifier,0.844907,0.828929,0.832233,0.839916,0.837801,0.839916


### Candidate VS London

In [40]:
df = featuresFactory.burchard_by_london_VS_london()

n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram


In [41]:
burchard_by_london_VS_zwickau_models_experiment = thesisModelFeatures.ModelsExperiment(df, 'big_hope_2')

In [42]:
burchard_by_london_VS_zwickau_models_experiment.run_cross_validate()

running: SVM_linear


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

Elapsed time to compute the cross validate: 0.863 minutes
running: SVM_RBF
Elapsed time to compute the cross validate: 0.928 minutes
running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.158 minutes
running: QuadraticDiscriminantAnalysis


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analy

Elapsed time to compute the cross validate: 0.968 minutes
running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.160 minutes
running: GaussianProcessClassifier
Elapsed time to compute the cross validate: 1.014 minutes
running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.133 minutes
running: MLPClassifier
Elapsed time to compute the cross validate: 5.309 minutes
running: GaussianNB
Elapsed time to compute the cross validate: 0.078 minutes
running: KNeighborsClassifier
Elapsed time to compute the cross validate: 0.071 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 0.752 minutes
running: XGBClassifier
Elapsed time to compute the cross validate: 2.358 minutes


In [43]:
burchard_by_london_VS_zwickau_models_experiment.show_cross_validate_results()

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
RandomForestClassifier,0.782091,0.726071,0.721159,0.750000,0.732690,0.750000
XGBClassifier,0.757065,0.729881,0.728194,0.747222,0.737307,0.747222
AdaBoostClassifier,0.690439,0.682649,0.681939,0.691667,0.688885,0.691667
MLPClassifier,0.692009,0.665238,0.662545,0.686111,0.674059,0.686111
SVM_RBF,0.793931,0.632649,0.597910,0.677778,0.621549,0.677778
GaussianProcessClassifier,0.799217,0.627946,0.587528,0.675000,0.612606,0.675000
DecisionTreeClassifier,0.662437,0.657440,0.655816,0.666667,0.663093,0.666667
GaussianNB,0.608815,0.568393,0.545137,0.602778,0.565224,0.602778
KNeighborsClassifier,0.589018,0.556905,0.538614,0.580556,0.554849,0.580556
SVM_linear,0.281944,0.500000,0.360526,0.563889,0.406725,0.563889


In [44]:
modelsToTest = [
    ( thesisModelFeatures.Model(RandomForestClassifier(max_depth = 11, n_estimators = 300, random_state = 0)), 'RandomForestClassifier', {} ),
    ( thesisModelFeatures.Model(AdaBoostClassifier(learning_rate = 0.5, n_estimators = 300)), 'AdaBoostClassifier', {} ),
    ]

In [45]:
import time
scores_df = pd.DataFrame(dtype=float)
scoring = ('precision_macro', 'recall_macro', 'f1_macro', 'f1_micro', 'f1_weighted', 'accuracy')

cross_validate_results = []
for m, classifier_name, greed_search_cv_params in modelsToTest:
    print(f'running: {classifier_name}')
    start_time = time.time()
    classifier_cross_validate_score = m.run_cross_validate(df)
    elapsed_time = time.time() - start_time
    print(f"Elapsed time to compute the cross validate: {elapsed_time/60:.3f} minutes")
    cross_validate_results.append(classifier_cross_validate_score)
    for s in scoring:
        scores_df.loc[classifier_name, s] = classifier_cross_validate_score[f'test_{s}'].mean()

running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.244 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 4.259 minutes


In [46]:
scores_df

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
RandomForestClassifier,0.817203,0.750744,0.749225,0.777778,0.760445,0.777778
AdaBoostClassifier,0.753814,0.734911,0.735220,0.750000,0.743263,0.750000


In [47]:
modelsToTest = [
    ( thesisModelFeatures.Model(xgb.XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.1, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=9, max_leaves=0, min_child_weight=3, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1)), 'XGBoost', {} ),
    ]

In [48]:
import time
scores_df = pd.DataFrame(dtype=float)
scoring = ('precision_macro', 'recall_macro', 'f1_macro', 'f1_micro', 'f1_weighted', 'accuracy')

cross_validate_results = []
for m, classifier_name, greed_search_cv_params in modelsToTest:
    print(f'running: {classifier_name}')
    start_time = time.time()
    classifier_cross_validate_score = m.run_cross_validate(df)
    elapsed_time = time.time() - start_time
    print(f"Elapsed time to compute the cross validate: {elapsed_time/60:.3f} minutes")
    cross_validate_results.append(classifier_cross_validate_score)
    for s in scoring:
        scores_df.loc[classifier_name, s] = classifier_cross_validate_score[f'test_{s}'].mean()

running: XGBoost
Elapsed time to compute the cross validate: 2.384 minutes


In [49]:
scores_df

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
XGBoost,0.77378,0.748839,0.748533,0.763889,0.756434,0.763889


### Burchard by Zwickau vs zwickau

In [50]:
df = featuresFactory.burchard_by_zwickau_VS_zwickau()

n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram


In [51]:
burchard_by_zwickau_VS_zwickau_models_experiment = thesisModelFeatures.ModelsExperiment(df, 'big_hope_2')

In [52]:
burchard_by_zwickau_VS_zwickau_models_experiment.run_cross_validate()

running: SVM_linear


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

Elapsed time to compute the cross validate: 0.749 minutes
running: SVM_RBF
Elapsed time to compute the cross validate: 0.792 minutes
running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.140 minutes
running: QuadraticDiscriminantAnalysis


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analy

Elapsed time to compute the cross validate: 0.849 minutes
running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.141 minutes
running: GaussianProcessClassifier
Elapsed time to compute the cross validate: 0.905 minutes
running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.122 minutes
running: MLPClassifier
Elapsed time to compute the cross validate: 4.890 minutes
running: GaussianNB
Elapsed time to compute the cross validate: 0.074 minutes
running: KNeighborsClassifier
Elapsed time to compute the cross validate: 0.070 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 0.688 minutes
running: XGBClassifier
Elapsed time to compute the cross validate: 2.161 minutes


In [53]:
burchard_by_zwickau_VS_zwickau_models_experiment.show_cross_validate_results()

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
XGBClassifier,0.830416,0.813095,0.817459,0.825546,0.823580,0.825546
AdaBoostClassifier,0.793551,0.779524,0.782360,0.791008,0.789328,0.791008
RandomForestClassifier,0.708522,0.674405,0.673645,0.699664,0.687398,0.699664
DecisionTreeClassifier,0.694488,0.692976,0.687576,0.693529,0.693166,0.693529
GaussianProcessClassifier,0.790594,0.624881,0.595643,0.682185,0.626282,0.682185
SVM_RBF,0.734043,0.626190,0.610001,0.673613,0.635210,0.673613
MLPClassifier,0.628634,0.574643,0.563159,0.616471,0.587647,0.616471
KNeighborsClassifier,0.648487,0.550714,0.491053,0.616134,0.532939,0.616134
GaussianNB,0.642063,0.532381,0.474211,0.595882,0.514955,0.595882
SVM_linear,0.290840,0.500000,0.367719,0.581681,0.427924,0.581681


In [57]:
from sklearn.tree import DecisionTreeClassifier
modelsToTest = [
    ( thesisModelFeatures.Model(DecisionTreeClassifier(criterion='entropy', max_depth=90)), 'DecisionTreeClassifier', {} ),
    ( thesisModelFeatures.Model(AdaBoostClassifier(learning_rate=1, n_estimators=500)), 'AdaBoostClassifier', {} ),
    ]

In [58]:
import time
scores_df = pd.DataFrame(dtype=float)
scoring = ('precision_macro', 'recall_macro', 'f1_macro', 'f1_micro', 'f1_weighted', 'accuracy')

cross_validate_results = []
for m, classifier_name, greed_search_cv_params in modelsToTest:
    print(f'running: {classifier_name}')
    start_time = time.time()
    classifier_cross_validate_score = m.run_cross_validate(df)
    elapsed_time = time.time() - start_time
    print(f"Elapsed time to compute the cross validate: {elapsed_time/60:.3f} minutes")
    cross_validate_results.append(classifier_cross_validate_score)
    for s in scoring:
        scores_df.loc[classifier_name, s] = classifier_cross_validate_score[f'test_{s}'].mean()

running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.123 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 6.498 minutes


In [59]:
scores_df

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
DecisionTreeClassifier,0.751820,0.746071,0.743597,0.751008,0.749969,0.751008
AdaBoostClassifier,0.833893,0.810952,0.815891,0.825378,0.822527,0.825378


In [60]:
modelsToTest = [
    ( thesisModelFeatures.Model(xgb.XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=3,
                monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1)), 'XGBoost', {} ),
    ]

In [61]:
import time
scores_df = pd.DataFrame(dtype=float)
scoring = ('precision_macro', 'recall_macro', 'f1_macro', 'f1_micro', 'f1_weighted', 'accuracy')

cross_validate_results = []
for m, classifier_name, greed_search_cv_params in modelsToTest:
    print(f'running: {classifier_name}')
    start_time = time.time()
    classifier_cross_validate_score = m.run_cross_validate(df)
    elapsed_time = time.time() - start_time
    print(f"Elapsed time to compute the cross validate: {elapsed_time/60:.3f} minutes")
    cross_validate_results.append(classifier_cross_validate_score)
    for s in scoring:
        scores_df.loc[classifier_name, s] = classifier_cross_validate_score[f'test_{s}'].mean()

running: XGBoost
Elapsed time to compute the cross validate: 2.069 minutes


In [62]:
scores_df

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
XGBoost,0.84939,0.829405,0.833681,0.842773,0.839898,0.842773


### Burchad by zwickau VS Lodon

In [63]:
df = featuresFactory.burchard_by_zwickau_VS_london()

n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram


In [64]:
burchard_by_zwickau_VS_london_models_experiment = thesisModelFeatures.ModelsExperiment(df, 'big_hope_2')

In [65]:
burchard_by_zwickau_VS_london_models_experiment.run_cross_validate()

running: SVM_linear


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

Elapsed time to compute the cross validate: 0.814 minutes
running: SVM_RBF
Elapsed time to compute the cross validate: 0.849 minutes
running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.144 minutes
running: QuadraticDiscriminantAnalysis


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analy

Elapsed time to compute the cross validate: 0.945 minutes
running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.143 minutes
running: GaussianProcessClassifier
Elapsed time to compute the cross validate: 0.997 minutes
running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.131 minutes
running: MLPClassifier
Elapsed time to compute the cross validate: 5.117 minutes
running: GaussianNB
Elapsed time to compute the cross validate: 0.077 minutes
running: KNeighborsClassifier
Elapsed time to compute the cross validate: 0.071 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 0.745 minutes
running: XGBClassifier
Elapsed time to compute the cross validate: 2.338 minutes


In [66]:
burchard_by_zwickau_VS_london_models_experiment.show_cross_validate_results()

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
RandomForestClassifier,0.785902,0.716250,0.708787,0.741667,0.721158,0.741667
XGBClassifier,0.741122,0.724732,0.724051,0.738889,0.732281,0.738889
AdaBoostClassifier,0.728461,0.723244,0.722036,0.727778,0.726689,0.727778
MLPClassifier,0.709717,0.672351,0.670150,0.694444,0.681570,0.694444
SVM_RBF,0.784403,0.628958,0.592497,0.675000,0.616803,0.675000
GaussianProcessClassifier,0.799119,0.627708,0.587600,0.675000,0.612687,0.675000
DecisionTreeClassifier,0.648370,0.646250,0.642345,0.652778,0.649223,0.652778
GaussianNB,0.602903,0.575506,0.551375,0.611111,0.572094,0.611111
KNeighborsClassifier,0.589629,0.555446,0.535763,0.577778,0.551626,0.577778
SVM_linear,0.281944,0.500000,0.360526,0.563889,0.406725,0.563889


In [67]:
modelsToTest = [
    ( thesisModelFeatures.Model(RandomForestClassifier(max_depth=9, n_estimators=500, random_state=0)), 'RandomForestClassifier', {} ),
    ( thesisModelFeatures.Model(AdaBoostClassifier(learning_rate=0.5, n_estimators=300)), 'AdaBoostClassifier', {} ),
    ]

In [68]:
import time
scores_df = pd.DataFrame(dtype=float)
scoring = ('precision_macro', 'recall_macro', 'f1_macro', 'f1_micro', 'f1_weighted', 'accuracy')

cross_validate_results = []
for m, classifier_name, greed_search_cv_params in modelsToTest:
    print(f'running: {classifier_name}')
    start_time = time.time()
    classifier_cross_validate_score = m.run_cross_validate(df)
    elapsed_time = time.time() - start_time
    print(f"Elapsed time to compute the cross validate: {elapsed_time/60:.3f} minutes")
    cross_validate_results.append(classifier_cross_validate_score)
    for s in scoring:
        scores_df.loc[classifier_name, s] = classifier_cross_validate_score[f'test_{s}'].mean()

running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.329 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 4.189 minutes


In [69]:
scores_df

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
RandomForestClassifier,0.819718,0.746994,0.744779,0.775,0.756322,0.775
AdaBoostClassifier,0.776388,0.763780,0.763986,0.775,0.770504,0.775


In [70]:
modelsToTest = [
    ( thesisModelFeatures.Model(xgb.XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.1, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
               monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1)), 'XGBoost', {} ),
    ]

In [73]:
import time
scores_df = pd.DataFrame(dtype=float)
scoring = ('precision_macro', 'recall_macro', 'f1_macro', 'f1_micro', 'f1_weighted', 'accuracy')

cross_validate_results = []
for m, classifier_name, greed_search_cv_params in modelsToTest:
    print(f'running: {classifier_name}')
    start_time = time.time()
    classifier_cross_validate_score = m.run_cross_validate(df)
    elapsed_time = time.time() - start_time
    print(f"Elapsed time to compute the cross validate: {elapsed_time/60:.3f} minutes")
    cross_validate_results.append(classifier_cross_validate_score)
    for s in scoring:
        scores_df.loc[classifier_name, s] = classifier_cross_validate_score[f'test_{s}'].mean()

running: XGBoost
Elapsed time to compute the cross validate: 2.349 minutes


In [74]:
scores_df

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
XGBoost,0.763781,0.748155,0.747526,0.763889,0.756159,0.763889
